In [21]:
import pandas as pd
from nba_api.stats.endpoints import leaguedashplayerstats, leaguedashteamclutch, leaguegamelog, boxscoreplayertrackv2, playercareerstats, teamplayerdashboard, leaguestandings
from nba_api.stats.static import players, teams
import time
pd.set_option('display.max_columns', None)
from IPython import display as ICD
import warnings
from datetime import date
import pickle
warnings.filterwarnings('ignore')

In [22]:
abbrv_mapping = {'DEN':"1610612743", 
           'SAS':"1610612759", 
           'IND':"1610612754", 
           'MEM':"1610612763", 
           'NOP':"1610612740", 
           'CHI':"1610612741", 
           'MIN':"1610612750", 
           'MIA':"1610612748", 
           'ATL':"1610612737",
           'ORL':"1610612753", 
           'SAC':"1610612758", 
           'UTA':"1610612762", 
           'CHA':"1610612766", 
           'WAS':"1610612764", 
           'HOU':"1610612745", 
           'PHX':"1610612756", 
           'OKC':"1610612760", 
           'POR':"1610612757",
           'LAC':"1610612746", 
           'GSW':"1610612744", 
           'PHI':"1610612755", 
           'LAL':"1610612747", 
           'BKN':"1610612751", 
           'TOR':"1610612761", 
           'BOS':"1610612738", 
           'MIL':"1610612749", 
           'DET':"1610612765",
           'CLE':"1610612739", 
           'NYK':"1610612752",
           'DAL':"1610612742"}

name_mapping = {'Nuggets':"1610612743", 
           'Spurs':"1610612759", 
           'Pacers':"1610612754", 
           'Memphis':"1610612763", 
           'Pelicans':"1610612740", 
           'Bulls':"1610612741", 
           'Timberwolves':"1610612750", 
           'Heat':"1610612748", 
           'Hawks':"1610612737",
           'Magic':"1610612753", 
           'Kings':"1610612758", 
           'Jazz':"1610612762", 
           'Hornets':"1610612766", 
           'Wizards':"1610612764", 
           'Rockets':"1610612745", 
           'Suns':"1610612756", 
           'Thunder':"1610612760", 
           'Trail Blazers':"1610612757",
           'Clippers':"1610612746", 
           'Warriors':"1610612744", 
           '76ers':"1610612755", 
           'Lakers':"1610612747", 
           'Nets':"1610612751", 
           'Raptors':"1610612761", 
           'Celtics':"1610612738", 
           'Bucks':"1610612749", 
           'Pistons':"1610612765",
           'Cavaliers':"1610612739", 
           'Knicks':"1610612752",
           'Mavericks':"1610612742"}

In [23]:
def clean_all_playoff_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season_type_all_star='Playoffs', season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

def clean_all_games(season_name):
    '''
    clean_all_playoff_games - add a few columns and remove irrelevant ones
    '''
    all_games = leaguegamelog.LeagueGameLog(season=season_name).get_data_frames()[0]
    all_games['GAME_DATE'] = pd.to_datetime(all_games['GAME_DATE']) 
    all_games = all_games.sort_values('GAME_DATE',ascending = False)
    all_games['OpponentTeamID'] = all_games['MATCHUP'].apply(lambda x: int(abbrv_mapping[x[-3:]]))
    all_games['OPTS'] = all_games['PTS'] - all_games['PLUS_MINUS']
    all_games = all_games[['GAME_ID','GAME_DATE','TEAM_ID','OpponentTeamID','MATCHUP','PTS','OPTS']]
    return all_games.rename(columns = {'GAME_ID':'GameID','GAME_DATE':'GameDate', 'TEAM_ID':'TeamID'})

all_playoff_games = clean_all_playoff_games('2019-20')
all_regseason_playoff_games = clean_all_games('2019-20')
all_reg_season_playoff_games = pd.concat([all_playoff_games, all_regseason_playoff_games]).sort_values('GameDate',ascending = False)
all_reg_season_playoff_games

,GameID,GameDate,TeamID,OpponentTeamID,MATCHUP,PTS,OPTS
83,0041900176,2020-08-31,1610612745,1610612760,HOU @ OKC,100,104
80,0041900201,2020-08-31,1610612748,1610612749,MIA @ MIL,115,104
82,0041900176,2020-08-31,1610612760,1610612745,OKC vs. HOU,104,100
81,0041900201,2020-08-31,1610612749,1610612748,MIL vs. MIA,104,115
74,0041900166,2020-08-30,1610612743,1610612762,DEN @ UTA,119,107
...,...,...,...,...,...,...,...
15,0021900007,2019-10-23,1610612748,1610612763,MIA vs. MEM,120,101
2,0021900001,2019-10-22,1610612761,1610612740,TOR vs. NOP,130,122
1,0021900002,2019-10-22,1610612747,1610612746,LAL @ LAC,102,112
3,0021900001,2019-10-22,1610612740,1610612761,NOP @ TOR,122,130


In [24]:
player_rotation = pd.read_csv('players_rotation.csv')
players_df = pd.read_csv('players.csv')
first_game_missing_mins = player_rotation.groupby('TEAM_ID').sum()['MIN'] / players_df.groupby('TEAM_ID').sum()['MIN']

In [25]:
def missing_mins(curr_game_id, team_id, season_name):
    '''
    missing_mins: gets the fraction of the season's minutes played by injured/inactive/traded players
    '''
    time.sleep(0.5)
    # get the season stats of the team
    season_stats = teamplayerdashboard.TeamPlayerDashboard(team_id, season=season_name).get_data_frames()[1]
    
    # get the game currently being assessed
    curr_game = boxscoreplayertrackv2.BoxScorePlayerTrackV2(game_id=curr_game_id).get_data_frames()[0]
    
    player_game_stats = curr_game[curr_game['TEAM_ID'] == int(team_id)]
    combined = pd.merge(season_stats, player_game_stats, how='left', on='PLAYER_ID')
    
    total_mins = combined['MIN_x'].sum()
    missing_mins = combined[(combined['MIN_y'] == "0:00") | (combined['MIN_y'].isna())]['MIN_x'].sum()
    return (total_mins - missing_mins) / total_mins

In [34]:
games = pd.read_csv('predictions/games.csv')
games['Date'] = games['Date'].astype('datetime64')
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,41900161.0,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,41900111.0,1.610613e+09,1.610613e+09,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,41900121.0,1.610613e+09,1.610613e+09,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,41900151.0,1.610613e+09,1.610613e+09,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,41900101.0,1.610613e+09,1.610613e+09,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,41900131.0,1.610613e+09,1.610613e+09,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,41900171.0,1.610613e+09,1.610613e+09,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,41900141.0,1.610613e+09,1.610613e+09,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,41900162.0,1.610613e+09,1.610613e+09,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,41900112.0,1.610613e+09,1.610613e+09,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [35]:
def get_gameid(row):
    if(not pd.isna(row['GameID'])):
        return row['GameID']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = all_playoff_games[all_playoff_games['OpponentTeamID'] == row['OpponentTeamID']]
    temp = temp[temp['GameDate'].apply(lambda x: x.date() == row['Date'].date())]
    if(len(temp) != 0):
        return temp['GameID'].iloc[0]
    return row['GameID']
def get_game_info(row, attribute, alt_attribute):
    if(pd.isna(row['GameID'])):
        return row[alt_attribute]
    try:
        return all_playoff_games[(all_playoff_games['GameID'] == row['GameID']) & (all_playoff_games['TeamID'] == row['TeamID'])][attribute].iloc[0]
    except:
        return row[alt_attribute]
def get_game_num(row):
    if(not pd.isna(row['GameNum'])):
        return row['GameNum']
    temp = all_playoff_games[all_playoff_games['TeamID'] == row['TeamID']]
    temp = temp[temp['OpponentTeamID'] == row['OpponentTeamID']]
    return len(temp) + 1

games['TeamID'] = games['TeamName'].apply(lambda x: int(name_mapping[x]))
games['OpponentTeamID'] = games['OpponentTeamName'].apply(lambda x: int(name_mapping[x]))
games['GameID'] = games.apply(lambda x: get_gameid(x), axis = 1)
games['Date'] = games.apply(lambda x: get_game_info(x, 'GameDate', 'Date'), axis = 1)
games['TeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'PTS', 'TeamPointsActual'), axis = 1)
games['OpponentTeamPointsActual'] = games.apply(lambda x: get_game_info(x, 'OPTS', 'OpponentTeamPointsActual'), axis = 1)
games['GameNum'] = games.apply(lambda x: get_game_num(x), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [36]:
def get_series_lead(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series)
    games_won = len(series[series['PTS'] > series['OPTS']])
    return 2 * games_won - len_series

def get_series_win_percent(games, curr_game):
    '''
    gets_series_length - get the game number of the series
    '''
    series = games[(games['TeamID'] == curr_game['TeamID']) & (games['OpponentTeamID'] == curr_game['OpponentTeamID'])]
    len_series = len(series) + 1
    games_won = len(series[series['PTS'] > series['OPTS']])
    return games_won/len_series

In [37]:
# creating the games dataframe
games_copy = games[pd.isna(games['TeamWinProba'])][['Date','TeamID', 'OpponentTeamID']]
games_copy_reversed = games_copy.copy()
games_copy_reversed['TeamID'] = games_copy['OpponentTeamID']
games_copy_reversed['OpponentTeamID'] = games_copy['TeamID']

all_games = pd.concat([games_copy, games_copy_reversed]).sort_index().reset_index(drop=True)

reg_season_ratings = leaguestandings.LeagueStandings(season='2019-20', season_type = 'Regular Season').get_data_frames()[0]
reg_season_ratings = reg_season_ratings[['TeamID','PointsPG','OppPointsPG']]

#ratings
all_games = pd.merge(all_games, reg_season_ratings, how='left', on='TeamID')
all_games = pd.merge(all_games, reg_season_ratings, how='left', left_on='OpponentTeamID', right_on='TeamID', suffixes=['','Opponent'])

all_games['SeriesGP'] = all_games.apply(lambda x: 1 + len(all_playoff_games[(all_playoff_games['TeamID'] == x['OpponentTeamID']) & (all_playoff_games['OpponentTeamID'] == x['TeamID'])]), axis = 1) 
all_games['SeriesLead'] = all_games.apply(lambda x: get_series_lead(all_playoff_games, x), axis = 1) 
all_games['SeriesWinPercent'] = all_games.apply(lambda x: get_series_win_percent(all_playoff_games, x), axis = 1) 





all_games['PrevPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['PTS'].iloc[0], axis = 1) 
all_games['PrevOPTS'] = all_games.apply(lambda x: all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['OPTS'].iloc[0], axis = 1) 
all_games['PrevMargin'] = all_games['PrevPTS'] - all_games['PrevOPTS']


all_games['MissingMins'] = all_games.apply(lambda x: missing_mins(all_playoff_games[all_playoff_games['TeamID'] == x['TeamID']]['GameID'].iloc[0], x['TeamID'], '2019-20'), axis = 1)
# first_game_missing_mins_copy = first_game_missing_mins.to_frame().reset_index()
# first_game_missing_mins_copy.columns = ['TeamID', 'MissingMins']
# all_games = pd.merge(all_games, first_game_missing_mins_copy, how='left', on='TeamID')


playoff_experience = pd.read_csv('playoff_experience.csv')
all_games = pd.merge(all_games, playoff_experience, how = 'left', on = 'TeamID')
all_games


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins
0,2020-09-01,1610612761,1610612738,112.8,106.5,1610612738,113.7,107.3,2,-1,0.000000,94,112,-18,0.915800,397.0,13204.620000
1,2020-09-01,1610612738,1610612761,113.7,107.3,1610612761,112.8,106.5,2,1,0.500000,112,94,18,0.788130,379.0,5772.186667
2,2020-09-01,1610612743,1610612762,111.3,109.2,1610612762,111.3,108.8,7,0,0.428571,119,107,12,0.778814,391.0,6895.763333
3,2020-09-01,1610612762,1610612743,111.3,108.8,1610612743,111.3,109.2,7,0,0.428571,107,119,-12,0.749396,396.0,6328.350000
4,2020-09-02,1610612749,1610612748,118.7,108.6,1610612748,112.0,109.1,2,-1,0.000000,104,115,-11,0.719253,446.0,16320.056667
5,2020-09-02,1610612748,1610612749,112.0,109.1,1610612749,118.7,108.6,2,1,0.500000,115,104,11,0.785951,398.0,10360.451667
6,2020-09-02,1610612745,1610612760,117.8,114.8,1610612760,110.4,108.4,7,0,0.428571,100,104,-4,0.821282,434.0,16812.746667
7,2020-09-02,1610612760,1610612745,110.4,108.4,1610612745,117.8,114.8,7,0,0.428571,104,100,4,0.830964,381.0,7951.911667


In [38]:
# bias boosted monte carlo simulation

In [39]:
import random
import matplotlib.pyplot as plt
import numpy as np

from sklearn.linear_model import LinearRegression
def get_weight(x, col_value, range_val):
    return 1/(00.1+abs((x-col_value)/range_val))
def get_rand_sample(df, col_name, col_value, sample_col_name, N, range_val):
    weights = df[col_name].apply(lambda x: get_weight(x,col_value, range_val))
    to_return = random.choices(population=list(df[sample_col_name]), 
                               weights=weights,
                               k=N)
    return to_return

In [40]:
pts_model = pickle.load( open( "pts_model", "rb" ) )
opts_model = pickle.load( open( "opts_model", "rb" ) )
range_vals = pickle.load( open( "range_vals", "rb" ) )
playoff_data = pd.read_csv('all_playoff_games.csv')
offense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','PointsPG','OppPointsPGOpponent','PrevPTS']
defense_cols = ['PlayoffMins','Age','SeriesGP','SeriesLead', 'SeriesWinPercent', 'MissingMins','PrevMargin','OppPointsPG','PointsPGOpponent','PrevOPTS']

In [41]:
def get_predictions(playoff_games, cols, predictor, N, model = None):
    predictions = playoff_games[['Date', 'TeamID','OpponentTeamID']]
    col_sum = []
    for x in cols:
        col_sum.append(x + "Sum")
    for col in cols:
        predictions[col] = playoff_games.apply(lambda x: get_rand_sample(playoff_data,col,x[col],predictor,N,range_vals[col]), axis = 1)
    for col in range(len(cols)):
        predictions[col_sum[col]] = predictions[cols[col]].apply(lambda x: sum(x)/N)
    X = predictions[col_sum]
    
    coefs = pd.Series(model.coef_, index = col_sum).abs()
    # predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: (np.array(x.values.tolist()) * coefs.values[:, None]).sum(axis=0)/coefs.sum(), axis = 1)
    predictions[predictor + 'Combined'] = predictions[cols].apply(lambda x: model.predict(np.array(x.values.tolist()).transpose()), axis = 1)
    predictions[predictor + 'LinearRegression'] = model.predict(X)
    temp = predictions[['TeamID'] + col_sum + [predictor+'LinearRegression']]
    temp['Name'] = temp['TeamID'].apply(lambda x: list(name_mapping.keys())[list(pd.Series(list(name_mapping.values())).astype(int)).index(x)])
    ICD.display(temp)
    temp.to_csv('predictions_intermed/' + predictor+date.today().strftime('%d-%m-%Y') + '.csv', index = False)
    return predictions[[predictor + 'Combined', predictor + 'LinearRegression']]

In [42]:
pts_df = get_predictions(all_games, offense_cols, 'AdjustedPTS', 10000, pts_model)
opts_df = get_predictions(all_games, defense_cols, 'AdjustedOPTS', 10000, opts_model)
final = pd.concat([all_games, pts_df, opts_df], axis = 1)
final['WinProba'] = final.apply(lambda x: np.count_nonzero(x['AdjustedPTSCombined'] > x['AdjustedOPTSCombined'])/1000, axis = 1)
final['WinProbaScaled'] = final.apply(lambda x: x['WinProba']* 1/final[((final['SeriesGP'] == x['SeriesGP'])) & (((final['TeamID'] == x['TeamID']) & (final['OpponentTeamID'] == x['OpponentTeamID'])) | ((final['TeamID'] == x['OpponentTeamID']) & (final['OpponentTeamID'] == x['TeamID'])))]['WinProba'].sum(), axis = 1)
final['WinProba2x'] = final.apply(lambda x: max(0,min(1,2*x['WinProbaScaled']-0.5)), axis = 1)
ICD.display(final)
final = final[['Date','TeamID','OpponentTeamID','AdjustedPTSLinearRegression','AdjustedOPTSLinearRegression', 'WinProba2x']]

,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,PointsPGSum,OppPointsPGOpponentSum,PrevPTSSum,AdjustedPTSLinearRegression,Name
0,1610612761,105.435098,106.066631,107.401402,105.371368,106.013622,106.800289,105.571337,106.829025,106.092148,104.828339,107.261173,Raptors
1,1610612738,105.066801,106.090141,107.156495,106.425066,106.355828,106.037846,106.953955,107.505186,106.096895,107.194009,112.060080,Celtics
2,1610612743,105.258496,105.957636,104.066714,105.607240,105.122923,105.971804,106.797108,106.040058,105.805826,107.558564,102.392752,Nuggets
3,1610612762,105.164435,105.860511,104.130174,105.522743,105.049338,105.567922,105.784338,106.319294,106.161684,106.181296,99.445535,Jazz
4,1610612749,106.002026,106.093798,107.497731,105.201562,106.158732,105.605554,105.972340,108.476180,105.895289,105.853279,110.628960,Bucks
5,1610612748,104.907025,105.953665,107.428050,106.615143,106.324652,105.798347,106.713113,106.209974,105.912472,107.417477,109.842676,Heat
6,1610612745,105.908678,106.223180,103.762564,105.636176,105.339373,106.204713,105.781859,108.709834,105.963445,105.211549,102.358894,Rockets
7,1610612760,105.186704,106.113066,104.016234,105.629924,105.329560,106.218020,106.011643,106.534087,107.508755,105.630049,102.885814,Thunder


,TeamID,PlayoffMinsSum,AgeSum,SeriesGPSum,SeriesLeadSum,SeriesWinPercentSum,MissingMinsSum,PrevMarginSum,OppPointsPGSum,PointsPGOpponentSum,PrevOPTSSum,AdjustedOPTSLinearRegression,Name
0,1610612761,105.578304,106.055210,107.339888,106.308179,107.355946,106.631225,106.916129,105.849478,107.260028,106.801846,112.978585,Raptors
1,1610612738,106.690231,106.098940,107.132957,105.488500,105.171503,105.464485,105.688240,106.020197,106.722006,104.908772,106.292952,Celtics
2,1610612743,106.300977,106.129585,104.079158,105.446287,104.873389,105.395747,106.102279,106.303103,106.421494,105.940615,101.621192,Nuggets
3,1610612762,106.463615,106.037951,104.035101,105.610566,104.906454,105.659760,106.716300,106.002662,106.356222,107.545364,104.069678,Jazz
4,1610612749,105.476331,105.951748,107.466406,106.249513,107.152045,105.804302,106.367454,105.929942,106.743357,107.164209,110.312727,Bucks
5,1610612748,105.947833,105.996814,107.374559,105.176317,104.855181,105.299995,105.761837,105.973507,108.625103,105.612800,108.931931,Heat
6,1610612745,105.282326,106.150607,103.817960,105.566282,105.004301,105.485266,105.955931,107.206438,106.414843,105.680139,101.559292,Rockets
7,1610612760,106.200709,105.899308,103.946469,105.476100,104.600192,105.524330,105.794515,105.896660,108.597621,105.003650,102.866232,Thunder


,Date,TeamID,OpponentTeamID,PointsPG,OppPointsPG,TeamIDOpponent,PointsPGOpponent,OppPointsPGOpponent,SeriesGP,SeriesLead,SeriesWinPercent,PrevPTS,PrevOPTS,PrevMargin,MissingMins,Age,PlayoffMins,AdjustedPTSCombined,AdjustedPTSLinearRegression,AdjustedOPTSCombined,AdjustedOPTSLinearRegression,WinProba,WinProbaScaled,WinProba2x
0,2020-09-01,1610612761,1610612738,112.8,106.5,1610612738,113.7,107.3,2,-1,0.000000,94,112,-18,0.915800,397.0,13204.620000,"[89.12941264615279, 50.84751166837623, 162.752...",107.261173,"[245.8583856826683, 115.94127008135774, 124.80...",112.978585,4.675,0.469330,0.438661
1,2020-09-01,1610612738,1610612761,113.7,107.3,1610612761,112.8,106.5,2,1,0.500000,112,94,18,0.788130,379.0,5772.186667,"[131.84700862539557, 136.60875003584306, 131.8...",112.060080,"[86.06664730369016, 155.83059512309615, 148.17...",106.292952,5.286,0.530670,0.561339
2,2020-09-01,1610612743,1610612762,111.3,109.2,1610612762,111.3,108.8,7,0,0.428571,119,107,12,0.778814,391.0,6895.763333,"[117.42984502546892, 109.13924525707966, 76.73...",102.392752,"[167.40174603043965, 31.1053432007418, 155.963...",101.621192,5.098,0.516986,0.533972
3,2020-09-01,1610612762,1610612743,111.3,108.8,1610612743,111.3,109.2,7,0,0.428571,107,119,-12,0.749396,396.0,6328.350000,"[48.811227943694576, 136.66692615960142, 39.75...",99.445535,"[100.0366785031265, 98.08156658864596, 117.766...",104.069678,4.763,0.483014,0.466028
4,2020-09-02,1610612749,1610612748,118.7,108.6,1610612748,112.0,109.1,2,-1,0.000000,104,115,-11,0.719253,446.0,16320.056667,"[69.25194012289921, 57.411191174186115, 139.69...",110.628960,"[37.75655623476882, 98.33553673367169, 114.179...",110.312727,4.989,0.499700,0.499399
5,2020-09-02,1610612748,1610612749,112.0,109.1,1610612749,118.7,108.6,2,1,0.500000,115,104,11,0.785951,398.0,10360.451667,"[26.40249801256732, 120.48350845201162, 124.52...",109.842676,"[115.23587164955075, 128.98634174884, 100.3679...",108.931931,4.995,0.500300,0.500601
6,2020-09-02,1610612745,1610612760,117.8,114.8,1610612760,110.4,108.4,7,0,0.428571,100,104,-4,0.821282,434.0,16812.746667,"[123.4683916320796, 217.0338921860955, 93.0228...",102.358894,"[127.63169565727571, 12.554918828230711, 103.7...",101.559292,5.047,0.503341,0.506682
7,2020-09-02,1610612760,1610612745,110.4,108.4,1610612745,117.8,114.8,7,0,0.428571,104,100,4,0.830964,381.0,7951.911667,"[122.66750564908716, 158.92713517475772, 224.2...",102.885814,"[4.418265607149351, 161.67668951531186, 40.305...",102.866232,4.980,0.496659,0.493318


In [43]:
for x in range(1, len(final),2):
    temp = final['AdjustedPTSLinearRegression'].iloc[x]
    final['AdjustedPTSLinearRegression'].iloc[x] = final['AdjustedOPTSLinearRegression'].iloc[x]
    final['AdjustedOPTSLinearRegression'].iloc[x] = temp
final =final.groupby(np.arange(len(final))//2).agg({'Date': 'first',
                                            'TeamID':'first',
                                             'OpponentTeamID':'first',
                                             'AdjustedPTSLinearRegression':'mean',
                                             'AdjustedOPTSLinearRegression':'mean',
                                             'WinProba2x':'first'})

In [44]:
import scipy.stats
norm_dist = scipy.stats.norm(0, 10)
final['ProbaNorm'] = final.apply(lambda x: norm_dist.cdf(x['AdjustedPTSLinearRegression'] - x['AdjustedOPTSLinearRegression']), axis = 1)
final

,Date,TeamID,OpponentTeamID,AdjustedPTSLinearRegression,AdjustedOPTSLinearRegression,WinProba2x,ProbaNorm
0,2020-09-01,1610612761,1610612738,106.777062,112.519332,0.438661,0.282907
1,2020-09-01,1610612743,1610612762,103.231215,100.533363,0.533972,0.606337
2,2020-09-02,1610612749,1610612748,109.780446,110.077702,0.499399,0.488143
3,2020-09-02,1610612745,1610612760,102.612563,102.222553,0.506682,0.515555


In [45]:
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [46]:
def get_game_info_calculated(row, final_col, games_col):
    if(not pd.isna(row[games_col])):
        return row[games_col]
    temp = final[final['TeamID'] == row['TeamID']]
    temp = final[final['OpponentTeamID'] == row['OpponentTeamID']]
    return temp[final_col].iloc[0]

games['TeamWinProba'] = games.apply(lambda x: get_game_info_calculated(x, 'ProbaNorm', 'TeamWinProba'), axis = 1)
games['OpponentTeamWinProba'] = games.apply(lambda x: 1 - x['TeamWinProba'], axis = 1)
games['TeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedPTSLinearRegression', 'TeamPoints'), axis = 1)
games['OpponentTeamPoints'] = games.apply(lambda x: get_game_info_calculated(x, 'AdjustedOPTSLinearRegression', 'OpponentTeamPoints'), axis = 1)
games

,Date,GameID,TeamID,OpponentTeamID,TeamName,OpponentTeamName,TeamWinProba,OpponentTeamWinProba,TeamPoints,OpponentTeamPoints,TeamPointsActual,OpponentTeamPointsActual,GameNum,Unnamed: 13,Unnamed: 14
0,2020-08-17,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.446276,0.553724,104.907378,106.258136,135.0,125.0,1.0,NaN,NaN
1,2020-08-17,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.587735,0.412265,107.809764,105.592550,134.0,110.0,1.0,NaN,NaN
2,2020-08-17,4.19001e+07,1610612738,1610612755,Celtics,76ers,0.390522,0.609478,100.521847,103.301425,109.0,101.0,1.0,NaN,NaN
3,2020-08-17,4.19002e+07,1610612746,1610612742,Clippers,Mavericks,0.676566,0.323434,107.580625,102.999441,118.0,110.0,1.0,NaN,NaN
4,2020-08-18,4.19001e+07,1610612749,1610612753,Bucks,Magic,0.616207,0.383793,107.926797,104.971470,110.0,122.0,1.0,NaN,NaN
5,2020-08-18,4.19001e+07,1610612754,1610612748,Pacers,Heat,0.690723,0.309277,105.559310,100.580296,101.0,113.0,1.0,NaN,NaN
6,2020-08-18,4.19002e+07,1610612745,1610612760,Rockets,Thunder,0.493277,0.506723,104.115003,104.283525,123.0,108.0,1.0,NaN,NaN
7,2020-08-18,4.19001e+07,1610612747,1610612757,Lakers,Trail Blazers,0.581614,0.418386,108.093849,106.033605,93.0,100.0,1.0,NaN,NaN
8,2020-08-19,4.19002e+07,1610612743,1610612762,Nuggets,Jazz,0.683257,0.316743,111.777117,107.008869,105.0,124.0,2.0,NaN,NaN
9,2020-08-19,4.19001e+07,1610612761,1610612751,Raptors,Nets,0.927164,0.072836,118.184497,103.634586,104.0,99.0,2.0,NaN,NaN


In [20]:
today = date.today()

pd.read_csv('predictions/games.csv').to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_before.csv', index = False)
games.to_csv('predictions/games.csv', index = False)
games.to_csv('predictions/games_' + today.strftime("%d-%m-%Y") +'_after.csv', index = False)